<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #9: Recommendation System

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">andrey.saitgalin@gmail.com</span>, 

<span style="color:blue; font-size: 12pt">Alexey Dral </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">aadral@gmail.com</span>

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
<a href= "https://github.com/ml-mipt/ml-mipt-part2/tree/master/2017/lectures">Лекции по курсу</a>

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_491_task9.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``

** Дедлайн**:
<span style="color:red; font-size: 12pt">16.12.2017 23:59 </span>

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Check Questions (0%)</h1> 
Вопросы отсутствуют - cool!

<h1 align="center"> Practical tasks (100%)</h1>


# Построение content-based рекомендательной системы
## Part 1
Для каждого фильма получите описание в текстовом виде с IMDB или (и) The movie database (у них есть открытое API). https://www.themoviedb.org/?language=ru. Вывести описание фильма под id 778. Также можно использовать рецензии.

In [1]:
import pandas as pd
import numpy as np
import tmdbsimple as tmdb
from scipy.spatial.distance import cosine
from tqdm import tqdm
import pickle 
from collections import Counter
from sklearn.model_selection import train_test_split

In [2]:
with open("keys.cfg","r") as f:
    tmdb.API_KEY = f.read()

In [3]:
movie = tmdb.Movies(778)
overview = movie.info()['overview']
title = movie.title
print(title)
print(overview)

Monsieur  Hulot's Holiday
Monsieur Hulot, Jacques Tati’s endearing clown, takes a holiday at a seaside resort, where his presence provokes one catastrophe after another. Tati’s masterpiece of gentle slapstick is a series of effortlessly well-choreographed sight gags involving dogs, boats, and firecrackers; it was the first entry in the Hulot series and the film that launched its maker to international stardom.


In [4]:
import time
def read_films(ids, sleep_time=0.00001):
    film_ids, titles, overviews = [],[],[]
    for i in ids:
        time.sleep(sleep_time)
        try:
            movie = tmdb.Movies(i)
            info = movie.info()
            title = info['title']
            overview = info['overview']
        except:
            continue
        titles.append(title)
        overviews.append(overview)
        film_ids.append(i)
    dict_to_save = {'film_ids':film_ids, 'titles':titles, 'overviews':overviews}
    np.save('uploaded_films_{}_{}.npy'.format(ids[0],ids[-1]) , dict_to_save)
    return film_ids, titles, overviews

In [5]:
# read_films(range(20000,35000))

In [6]:
!ls

Untitled.ipynb                 uploaded_films_1000_4999.npy
keys.cfg                       uploaded_films_200_209.npy
ml-20m                         uploaded_films_210_999.npy
movie_data.npy                 uploaded_films_5000_9999.npy
pickle_movies.pkl              zotov_497_task9.ipynb
uploaded_films_10000_19999.npy


In [7]:
# LOAD FILES
FNAMES = [
    'uploaded_films_10000_19999.npy', 
    'uploaded_films_5000_9999.npy',
    'uploaded_films_1000_4999.npy',
    'uploaded_films_210_999.npy',
    'uploaded_films_200_209.npy',
]
def read_files_1():
    ids = []
    descriptions = []
    titles = []
    for f in FNAMES:
        d = np.load(f).item()
        ids += d['film_ids']
        descriptions += d['overviews']
        titles += d['titles']
    return ids, descriptions, titles

def read_files_2():
    d = np.load('pickle_movies.pkl')
    keys = list(d.keys())
    descriptions = [d[k]['overview'] for k in keys]
    titles = [d[k]['title'] for k in keys]
    return keys, descriptions, titles
    
def build_mapping(ids):
    val_to_i, i_to_val = {},{}
    for index, value in enumerate(ids):
        i_to_val[index] = value
        val_to_i[value] = index
    return val_to_i, i_to_val


In [8]:
ids, descriptions, titles = read_files_2()
val_to_i, i_to_val = build_mapping(ids)
# print(len(val_to_i) , len(i_to_val))
assert len(val_to_i) == len(i_to_val)
assert len(ids) == len(descriptions) and len(ids) == len(titles)
len(ids)

180283

In [9]:
!ls ml-20m

README.txt        genome-tags.csv   movies.csv        tags.csv
genome-scores.csv links.csv         ratings.csv


## Part 2 
Извлеките текстовые признаки у фильмов: tf idf, биграммы (может какие-то еще). Биграммы - статистика по биграммам, сколько какая биграмма встречается. Формулы для TF-IDF смотреть тут: https://en.wikipedia.org/wiki/Tf%E2%80%93idf 
    1. Используйте TF с double 0.5 normalization
    2. Используйте стандартный IDF: 
$$idf(t, D) = \frac{log N}{(1 + |d in D : t in d|)}$$

In [10]:
# put your code here
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(descriptions)

In [11]:
tf_idf.shape

(180283, 182571)

## Part 3 
Выведите подсчитанные признаки из пункта 2 для фильма с id 99114.

In [12]:
try:
    movie = tmdb.Movies(99114)
    movie.info()
except Exception:
    print('Что это за фильм?')

Что это за фильм?


In [13]:
# ВОТ ОН 68718
# mid_to_tmd[99114] == 68718
row = tf_idf.getrow(val_to_i[68718])# put your code here
print(titles[val_to_i[68718]])
print(descriptions[val_to_i[68718]])
Counter(np.array(row.todense())[0])

Django Unchained
With the help of a German bounty hunter, a freed slave sets out to rescue his wife from a brutal Mississippi plantation owner.


Counter({0.0: 182551,
         0.054226901019748426: 1,
         0.060186406982517274: 1,
         0.063990905656852295: 1,
         0.086584249974183944: 1,
         0.090929609699591615: 1,
         0.10225842812652504: 1,
         0.13509367380798046: 1,
         0.17687076361233151: 1,
         0.17759828473771597: 1,
         0.22104827310527214: 1,
         0.23449642910833335: 1,
         0.2404058396076586: 1,
         0.24563297889689367: 1,
         0.25057658941997746: 1,
         0.26855732438532576: 1,
         0.29408933912724339: 1,
         0.31114951364901228: 1,
         0.32687019747671314: 1,
         0.34166149708111643: 1,
         0.34390885849298591: 1})

## Part 4
Для каждого пользователя найдите топ жанров, которые нравятся пользователю (можно делать как на семинаре top 5, top 3 и top 2) и порекомендуйте таким пользователя фильмы близкие по текстовому описанию (см. пункт 2). Для оценки результатов пункта 3 используйте метрику HitRate и метрики ранжирования (N)DCG. Формулу для подсчета HitRate можно взять на семинаре,  (N)DCG: https://en.wikipedia.org/wiki/Discounted_cumulative_gain



In [14]:
links = pd.read_csv('ml-20m/links.csv')
links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [15]:
tmd_to_mid = dict(zip(links['tmdbId'], links['movieId']))
mid_to_tmd = dict(zip(links['movieId'], links['tmdbId']))

In [16]:
mid_to_tmd[99114]

68718.0

In [17]:
movies = pd.read_csv('ml-20m/movies.csv')
# should be FIGHT CLUB
movies[movies['movieId'] == tmd_to_mid[550]]

,movieId,title,genres
2873,2959,Fight Club (1999),Action|Crime|Drama|Thriller


In [18]:
ratings = pd.read_csv('ml-20m/ratings.csv')

In [19]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


## Genres

Будем считать метрику $\mathbb{GENRE--TF-IDF}$ популярности жанра у пользователя как :

$R = \frac{\sum_{\text{film} \in \text{genre}} \text{rating(film)}}{\log (\text{number of films with this genre})}$

In [20]:
genres_counter = Counter('|'.join(movies['genres']).split('|'))
genres = list(genres_counter.keys())
print(genres)

['Film-Noir', 'Documentary', 'Adventure', 'Action', 'Crime', 'Fantasy', 'Comedy', 'Horror', 'Mystery', 'Drama', 'Animation', 'Sci-Fi', 'IMAX', 'Children', 'Western', 'Thriller', 'Romance', 'Musical', '(no genres listed)', 'War']


In [21]:
movie_to_genres = {}
for i in range(movies.shape[0]):
    row = movies.iloc[i]
    movie_to_genres[row['movieId']] = row['genres'].split('|')

In [22]:
interesting_users = [1,2,3,10,20,666]
user_genres_rating = {}
for user in interesting_users:
    df = ratings[ratings['userId'] == user]
    for i in tqdm(range(df.shape[0]),position=0):
        row = df.iloc[i]
        user_id = int(row['userId'])
        movie_id = int(row['movieId'])
        rating = int(row['rating'])
        movie_genres = movie_to_genres[movie_id]
        if user_id not in user_genres_rating:
            user_genres_rating[user_id] = dict(zip(genres,list(np.zeros(len(genres)))))
        for genre in movie_to_genres[movie_id]:
            user_genres_rating[user_id][genre] += rating
for user in user_genres_rating.keys():
    for genre in user_genres_rating[user].keys():
        user_genres_rating[user][genre] /= np.log(genres_counter[genre])

100%|██████████| 68/68 [00:00<00:00, 5282.40it/s]


In [23]:
def get_top_genres(user_id ,n=5):
    d = user_genres_rating[user_id]
    scores = [(d[k],k) for k in d.keys()]
    return sorted(scores,reverse=True)[:n]
    
print(get_top_genres(1))
print(get_top_genres(3))
print(get_top_genres(666))

[(34.331746395029093, 'Fantasy'), (34.050481751173535, 'Adventure'), (28.654641550454755, 'Action'), (20.082647546334464, 'Horror'), (18.4903238872771, 'Sci-Fi')]
[(49.843481783094788, 'Sci-Fi'), (30.736389013521983, 'Action'), (27.214589581430364, 'Adventure'), (25.792671637206286, 'Drama'), (25.546956923122973, 'Thriller')]
[(13.054250134749305, 'Drama'), (10.959951185649791, 'Comedy'), (8.6415995353357076, 'Adventure'), (8.3269898522689036, 'Action'), (7.4471736868803804, 'Romance')]


# Рекомендации

Посчитаем для каждого фильма из списка предложенных к просмотру конкретному пользователю (еще не просмотренные фильмы) метрику $\textbf{score}(a)$: 
$$
\textbf{score}(a) = \frac{1}{N} \sum_{i,b=\text{watched}(i)}^{N} [ \text{tfidf_cosine_score}(a, b) * \text{rating}(b) ]
$$

На словах - взвешенный по рейтингу усредненный cosine_similarity_score по всем просмотренным фильмам. 

In [24]:
def get_recommendation_for_user(watched_films_id, rating, suggesting_list_ids):
    watched_tf_idf = tf_idf[watched_films_id]
    to_suggest_tf_idf = tf_idf[suggesting_list_ids]
    suggested_score = np.array([np.mean([tfidf_cosine_similarity(a,b) 
                                    for a in watched_tf_idf] * rating)
                           for b in to_suggest_tf_idf])
    sorted_args = np.flip(np.argsort(suggested_score),axis=-1)
    return sorted_args, suggested_score[sorted_args]

def tfidf_cosine_similarity(a, b):
    a = a.todense()
    a = np.ravel(a)
    b = b.todense()
    b = np.ravel(b)
    try:
        return -(cosine(a, b) - 1)
    except:
        pass
    return np.nan

### Построим рекомендации для нескольких пользователей и посчиатем NDCG

In [25]:
def process_recommendations(user_id = 1):
    user_ratings = ratings[ratings['userId'] == user_id]
    train, test = train_test_split(user_ratings,test_size=0.5)

    result_id = [],[]
    result_ratings = [],[]

    indices_list = [train['movieId'], test['movieId']]
    ratings_list = [train['rating'].values, test['rating'].values]

    for j in range(2):
        for n, i in enumerate(indices_list[j]):
            idx = int(mid_to_tmd[i])
            if idx in val_to_i.keys():
                result_id[j].append(val_to_i[idx])
                result_ratings[j].append(ratings_list[j][n])

    train_indices, test_indices = np.array(result_id[0]) , np.array(result_id[1])
    train_ratings, test_ratings = np.array(result_ratings[0]), np.array(result_ratings[1])
    indices, scores = get_recommendation_for_user(train_indices,
                                                train_ratings, 
                                                test_indices)
    recommended_indices = test_indices[indices]
    real_ratings = test_ratings[indices]
    
    return recommended_indices, real_ratings

Реализацию NDCG возьмем отсюда:

https://gist.github.com/bwhite/3726239


In [26]:

def dcg_at_k(r, k, method=0):
    """Score is discounted cumulative gain (dcg)

    Relevance is positive real values.  Can use binary
    as the previous methods.

    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1)
    3.0
    >>> dcg_at_k(r, 1, method=1)
    3.0
    >>> dcg_at_k(r, 2)
    5.0
    >>> dcg_at_k(r, 2, method=1)
    4.2618595071429155
    >>> dcg_at_k(r, 10)
    9.6051177391888114
    >>> dcg_at_k(r, 11)
    9.6051177391888114

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]

    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    """Score is normalized discounted cumulative gain (ndcg)

    Relevance is positive real values.  Can use binary
    as the previous methods.

    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4)
    0.9203032077642922
    >>> ndcg_at_k(r, 4, method=1)
    0.96519546960144276
    >>> ndcg_at_k([0], 1)
    0.0
    >>> ndcg_at_k([1], 2)
    1.0

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]

    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [27]:
%%time
recommended_indices, real_ratings = process_recommendations(1)
print(real_ratings)
print('NDCG at 5 = ', ndcg_at_k(real_ratings, 5))

[ 4.   5.   3.5  3.5  3.5  4.   4.   3.   3.5  4.   3.5  4.   4.   3.5  5.
  3.5  3.5  4.   4.   3.   4.   3.5  4.   4.   4.   4.   3.5  4.   4.   3.5
  3.5  4.   4.   3.   3.5  4.   3.5  4.   3.5]
NDCG at 5 =  0.890388000511
CPU times: user 2.23 s, sys: 45.9 ms, total: 2.27 s
Wall time: 2.3 s


In [28]:
%%time
recommended_indices, real_ratings = process_recommendations(2)
print(real_ratings)
print('NDCG at 5 = ', ndcg_at_k(real_ratings, 5))

[ 4.  5.  4.  5.  4.  5.  5.  5.  3.  2.  3.  3.  5.]
NDCG at 5 =  0.88423165666
CPU times: user 338 ms, sys: 11.3 ms, total: 350 ms
Wall time: 354 ms


In [29]:
%%time
recommended_indices, real_ratings = process_recommendations(3)
print(real_ratings)
print('NDCG at 5 = ', ndcg_at_k(real_ratings, 5))

[ 4.  3.  5.  4.  4.  5.  4.  3.  5.  4.  5.  4.  4.  5.  3.  4.  5.  4.
  5.  3.  5.  5.  3.  4.  3.  4.  5.  3.  2.  2.]
NDCG at 5 =  0.779275067813
CPU times: user 1.84 s, sys: 35.4 ms, total: 1.88 s
Wall time: 1.9 s


In [30]:
%%time
recommended_indices, real_ratings = process_recommendations(15)
print(real_ratings)
print('NDCG at 5 = ', ndcg_at_k(real_ratings, 5))

[ 3.  4.  4.  3.  4.  3.  4.  4.  2.  3.  3.  3.  2.  3.  3.  3.  3.  3.
  4.  3.  2.  3.  3.]
NDCG at 5 =  0.894710429175
CPU times: user 727 ms, sys: 15.2 ms, total: 742 ms
Wall time: 747 ms


Из примеров видно, что сортировка рейтингов происходит не очень качественно, особенно если пользователь поставил всем фильмам одинаково высокие оценки (например 4.5 - 5). Но все же, сильно плохие оценки (1 - 2) не попадают в рекомендации (топ 5 фильмов).